# 07 - Hydrogen Chain

**Overview** 

This notebook guides you through the simulation of the energy levels of an electron interacting with an infinite periodic chain of atoms. The notebook will adopt a semi-empirical approximation to describe the energy levels of a finite chain of atoms and will allow to extrapolate the results to the theoretical result for Bloch states. This will provide a visual connection between atomic energy states and band diagrams for a one dimensional system.

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
!pip install -q rdkit > /dev/null
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

In [ ]:
# @title Utility Functions { display-mode: "form" }
def tb_hamiltonian_1d(N, epsilon=0.0, t=-1.0, periodic=True):
    H = np.zeros((N, N), dtype=float)
    np.fill_diagonal(H, epsilon)
    off = t * np.ones(N - 1, dtype=float)
    H[np.arange(N-1), np.arange(1, N)] = off
    H[np.arange(1, N), np.arange(N-1)] = off
    if periodic and N > 2:
        H[0, -1] = t
        H[-1, 0] = t
    return H

def tb_dispersion_1d(k, epsilon=0.0, t=-1.0, a=1.0):
    return epsilon + 2.0 * t * np.cos(k * a)

def finite_k_open_chain(N, a=1.0):
    n = np.arange(1, N + 1)
    return n * np.pi / ((N + 1) * a)

def solve_finite_chain(N=20, epsilon=0.0, t=-1.0, periodic=True):
    H = tb_hamiltonian_1d(N, epsilon=epsilon, t=t, periodic=periodic)
    E, V = np.linalg.eigh(H)
    return E, V
import numpy as np

def _wrap_k(k, a=1.0):
    # map to (-pi/a, pi/a]
    return (k + np.pi/a) % (2*np.pi/a) - np.pi/a

def label_k_via_translation(E, V, a=1.0, energy_tol=1e-12):
    """
    Label periodic-chain eigenstates by signed k using the translation operator T.
    Also returns a rotated eigenvector matrix whose columns are chiral (traveling) states.

    Parameters
    ----------
    E : (N,) array of eigenvalues
    V : (N, N) eigenvectors (columns)
    a : lattice spacing
    energy_tol : tolerance to group degenerate energies

    Returns
    -------
    k : (N,) array of signed k in (-pi/a, pi/a]
    V_rot : (N, N) rotated eigenvectors with definite translation phase
    """
    N = V.shape[0]
    idx = np.argsort(E)
    E_sorted = E[idx]
    V_sorted = V[:, idx].copy()

    # group indices by (near-)degenerate energies
    groups = []
    start = 0
    for i in range(1, N+1):
        if i == N or not np.isclose(E_sorted[i], E_sorted[start], atol=energy_tol):
            groups.append((start, i))  # [start, i)
            start = i

    k_labels = np.empty(N, dtype=float)
    V_rot_sorted = V_sorted.copy().astype(complex)

    for (lo, hi) in groups:
        Vg = V_rot_sorted[:, lo:hi]          # N x g
        g = Vg.shape[1]

        # Build T in this subspace: S = <v_m | T | v_n> with (T psi)_j = psi_{j+1}
        # Implement T via roll by -1 along site axis
        TVg = np.roll(Vg, -1, axis=0)        # N x g
        S = Vg.conj().T @ TVg                # g x g

        # Diagonalize S: eigenvalues are e^{i k a}
        w, U = np.linalg.eig(S)
        # Rotate basis to chiral states
        Vg_rot = Vg @ U                      # N x g

        # Store rotated vectors back
        V_rot_sorted[:, lo:hi] = Vg_rot

        # Extract signed k from eigenvalues
        k_block = np.angle(w) / a            # in (-pi/a, pi/a]
        k_block = _wrap_k(k_block, a=a)

        # Pin exact special points
        # Gamma: keep exactly 0
        k_block[np.isclose(k_block, 0.0, atol=1e-14)] = 0.0
        # Zone edge (even N only): keep exactly +pi/a (convention)
        if N % 2 == 0:
            k_block[np.isclose(np.abs(k_block), np.pi/a, atol=1e-14)] = np.pi / a

        # Save
        k_labels[lo:hi] = k_block

    # undo sorting to match the original eigenvalue order
    inv = np.empty_like(idx)
    inv[idx] = np.arange(N)
    k_out = k_labels[inv]
    V_rot = V_rot_sorted[:, inv]

    return k_out, V_rot


# Group degenerate energies (same as in label_k_via_translation)
def group_degenerate_energies(E, tol=1e-12):
    idx = np.argsort(E)
    E_sorted = E[idx]
    groups = []
    start = 0
    for i in range(1, len(E)+1):
        if i == len(E) or not np.isclose(E_sorted[i], E_sorted[start], atol=tol):
            groups.append((start, i))  # [start, i)
            start = i
    return [idx[lo:hi] for (lo, hi) in groups]

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# 1s orbital (3D-normalized); sampled in the z=0 plane
def _phi_1s(r, a0=1.0):
    return (1.0/np.sqrt(np.pi)) * (a0 ** -1.5) * np.exp(-r / a0)

def _wrap_k(k, a=1.0):
    return (k + np.pi/a) % (2*np.pi/a) - np.pi/a

def _wavelength_from_k(k, a=1.0):
    if k is None or np.isclose(k, 0.0):
        return np.inf
    return 2*np.pi/abs(k)

def _bond_current(psi, t=-1.0, periodic=True):
    psi = psi.astype(np.complex128)
    N = psi.shape[0]
    j = np.zeros(N, dtype=float)
    if periodic:
        nxt = np.roll(psi, -1)
        j = 2.0 * np.imag(t * np.conj(psi) * nxt)
    else:
        nxt = np.zeros_like(psi)
        nxt[:-1] = psi[1:]
        j[:-1] = 2.0 * np.imag(t * np.conj(psi[:-1]) * nxt[:-1])
        j[-1] = 0.0
    return j

def visualize_state_card_periodic(
    E, V, index, *,
    a=1.0, t=-1.0, k=None, title_prefix=None,
    # LCAO ring geometry + rendering
    a0=0.6,             # 1s length scale (units of a)
    pad_radius=1.2,     # padding factor around ring radius in plot window
    nx=320, ny=320,     # xy grid resolution
    contour_levels=25,
    contour_log=False,
    contour_eps=1e-16,
    # Left info label
    left_label=None,    # str or None; if None, an automatic label is generated
    left_label_x=-0.5, # position in axes coords; negative => outside left
    left_label_y=0.5,   # vertical center (0..1) in axes coords
    left_label_rotation=0,  # rotate text so it reads vertically
    left_margin=0.22,   # increase left margin so label isn't clipped
):
    """
    Periodic chain visual: (1) amplitudes vs site, (2) LCAO 1s |Ψ(x,y)|^2 contour on a ring,
    (3) phase & bond current. An info label is placed just to the LEFT of the central plot.
    """
    N = V.shape[0]
    if not (0 <= index < N):
        raise IndexError(f"index must be in [0,{N-1}]")

    # Eigenvector coefficients on sites
    psi = V[:, index].astype(np.complex128)
    nrm = np.linalg.norm(psi)
    if nrm > 0: psi = psi / nrm
    coeffs = psi.copy()

    # Ring geometry: circumference = N*a => radius R = N a / (2π)
    R = N * a / (2.0 * np.pi)
    theta = 2.0 * np.pi * np.arange(N) / N
    centers_x = R * np.cos(theta)
    centers_y = R * np.sin(theta)

    # Build xy grid covering the ring with padding
    L = pad_radius * R
    x = np.linspace(-L, L, nx)
    y = np.linspace(-L, L, ny)
    X, Y = np.meshgrid(x, y, indexing='xy')

    # LCAO: Ψ(x,y) = Σ_j c_j φ_1s(|r - R_j|)
    Psi = np.zeros_like(X, dtype=np.complex128)
    for j_idx in range(N):
        dx = X - centers_x[j_idx]
        dy = Y - centers_y[j_idx]
        rj = np.sqrt(dx*dx + dy*dy)
        Psi += coeffs[j_idx] * _phi_1s(rj, a0=a0)
    rho_xy = np.real(np.conj(Psi) * Psi)  # |Ψ|^2

    # Metadata
    energy = float(E[index])
    k_show = None; lam = None; v = None
    if k is not None:
        k_wrapped = _wrap_k(float(k), a=a)
        if np.isclose(k_wrapped, 0.0): k_wrapped = 0.0
        if np.isclose(abs(k_wrapped), np.pi/a): k_wrapped = np.sign(k_wrapped)*(np.pi/a)
        k_show = k_wrapped
        lam = _wavelength_from_k(k_show, a=a)
        v = -2.0 * t * a * np.sin(k_show * a)  # dE/dk for NN TB (ħ=1)

    # 1D lines
    jaxis = np.arange(N)
    Re = np.real(psi); Im = np.imag(psi)
    phase = np.unwrap(np.angle(psi))
    jbond = _bond_current(psi, t=t, periodic=True)

    # ---- Figure ----
    fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=False,
                            gridspec_kw={'height_ratios': [2, 2.8, 2]})
    # make room on the left for the external label
    plt.subplots_adjust(left=left_margin)

    # (1) amplitudes
    axs[0].plot(jaxis, Re, marker='o', linewidth=1.1, label='Re ψ(j)')
    axs[0].plot(jaxis, Im, marker='o', linewidth=1.1, label='Im ψ(j)')
    axs[0].set_ylabel("Amplitude")
    axs[0].grid(True, linestyle='--', alpha=0.35)
    axs[0].legend(loc='best')
    axs[0].set_xlim(-0.5, N-0.5)

    # (2) contour of |Ψ(x,y)|^2 on the ring
    axc = axs[1]
    if contour_log:
        Z = np.clip(rho_xy, contour_eps, None)
        cntr = axc.contourf(X, Y, Z, levels=contour_levels, norm=LogNorm())
        cbar = fig.colorbar(cntr, ax=axc, pad=0.02); cbar.set_label(r"$|\Psi(x,y)|^2$ (log)")
    else:
        cntr = axc.contourf(X, Y, rho_xy, levels=contour_levels)
        cbar = fig.colorbar(cntr, ax=axc, pad=0.02); cbar.set_label(r"$|\Psi(x,y)|^2$")
    axc.scatter(centers_x, centers_y, s=20, marker='x', linewidths=1.2, label="sites")
    axc.set_aspect('equal', adjustable='box')
    axc.set_xlim(-L, L); axc.set_ylim(-L, L)
    axc.set_xlabel("x"); axc.set_ylabel("y")
    axc.legend(loc='upper right')
    axc.set_title("Periodic chain (ring) — 1s-LCAO density")

    # ---- LEFT-SIDE INFO LABEL, anchored to central axes ----
    if left_label is None:
        left_label = "\n".join([
            f"state {index}",
            f"E = {energy:.6f}",
            f"N = {N}, a = {a:.3g}",
            f"R = {R:.3g}, a0 = {a0:.3g}",
            (f"k = {k_show:.6f}" if k_show is not None else "k = —"),
            (f"λ = {'∞' if (k_show is not None and not np.isfinite(lam)) else (f'{lam:.3g}' if lam is not None else '—')}"),
            (f"v = {v:.3g}" if v is not None else "v = —"),
            f"grid {nx}×{ny}",
            "log scale" if contour_log else "linear scale",
        ])
    axc.text(
        left_label_x, left_label_y, left_label,
        transform=axc.transAxes,
        ha='right', va='center',
#        rotation=left_label_rotation,
        fontsize=9, family='monospace',
        bbox=dict(boxstyle="round", alpha=0.15, lw=0.5),
        clip_on=False,  # allow drawing in the margin
    )

    # (3) phase & current
    ax3 = axs[2]
    ax3.plot(jaxis, phase, marker='o', linewidth=1.1, label='arg ψ (unwrapped)')
    ax3.set_ylabel("Phase (rad)")
    ax3.grid(True, linestyle='--', alpha=0.35)
    ax3b = ax3.twinx()
    ax3b.plot(jaxis, jbond, marker='s', linewidth=1.1, label='bond current j_j', alpha=0.9)
    ax3b.set_ylabel("Current")
    lines, labels = ax3.get_legend_handles_labels()
    lines2, labels2 = ax3b.get_legend_handles_labels()
    ax3b.legend(lines + lines2, labels + labels2, loc='best')
    axs[2].set_xlabel("Site index j")
    axs[2].set_xlim(-0.5, N-0.5)

    # Title
    bits = []
    if title_prefix: bits.append(title_prefix)
    bits += [f"Periodic ring • state {index}", f"E = {energy:.6f}"]
    if k_show is not None:
        bits.append(f"k = {k_show:.6f}")
        bits.append(f"λ = {'∞' if not np.isfinite(lam) else f'{lam:.6f}'}")
        bits.append(f"v = {v:.6f}")
    fig.suptitle(" | ".join(bits), y=0.99)

    fig.tight_layout(rect=[0, 0.02, 1, 0.97])
    plt.show()


**Problem** 

In this notebook we want to characterize the energy levels of an electron interacting with an infinite 1D chain of atoms. This could correspond to a macroscopic one-dimensional system, such as a nanowire or a conjugated polymer. 

**Model**

The electronic states (energy levels) of an infinite periodic system can be expressed in Bloch form as 

$$ \psi_{\vec{k}}(\vec{r}) = e^{i\vec{k}\cdot\vec{r}} u_{\vec{k}}(\vec{r}) $$

where the function $u_{\vec{k}}(\vec{r})$ is identical in each lattice unit. 

For our problem we are considering a 1D lattice, where a generic lattice vector $\vec{R}$ can be expressed as 

$$\vec{R}_i \equiv R_i = a \cdot i$$

in terms of the lattice constant $a$ and an integer (positive or negative) number $i$. 

In order to simplify the quantum-mechanical treatement of the problem, and given the symmetry of the system, we will adopt a very semplified semi-empirical (a.k.a. Tight-Binding or TB) description, equivalent to the Huckel Hamiltonian. 

1) we will assume a minimal basis set composed by a single atomic orbital per lattice site
2) we will consider the basis set to be orthonormal, with no overlap between orbitals in nearby sites
3) we will only consider on-site terms (on site energy $\epsilon$ in TB notation, $\alpha$ for the Huckel model) and first nearest-neighbor terms (hopping term $-t$ in TB, $\beta$ in the Huckel model).

Given the above approximations and Bloch theorem, we only have one way to combine individual atomic orbitals. With only one orbital $f(r)$ per cell and all the translational symmetry of the system, the problem has no variational flexibility and the electronic states need to be of the form

$$\psi_{\vec{k}}(\vec{r}) = e^{i\vec{k}\cdot\vec{r}} \sum_{R_i} f(\vec{r}-\vec{R}_i) = \sum_{R_i} f(\vec{r}-\vec{R}_i) e^{i\vec{k}\cdot\vec{R}_i}  $$

We can thus use it to compute the corresponding expectation value of the energy of the system, namely:

$$\left<E\right> =  \left<\psi_{\vec{k}}\right|\hat{H}\left|\psi_{\vec{k}}\right> = \sum_{R_i}\sum_{R_j} \left<f(\vec{r}-\vec{R}_i)\right|\hat{H}\left|f(\vec{r}-\vec{R}_j) \right> e^{i\vec{k}\cdot(\vec{R}_j-\vec{R}_i)} $$

Given the three TB approximations above, each term in the $R_i$ summation is identical and, for each real-space lattice vector $R_i$, only three of the $R_j$ terms are different from zero. This allows to simplify the above expression into

$$\left<E\right> = \left( \alpha + \beta e^{i k \cdot a} + \beta e^{-i k \cdot a}\right) = \alpha + 2 \beta \cos(k a),  $$

where we implicitly chose to report the total energy per unit cell. 

The above solution holds for an infinite system and did not require us to diagonalize any Hamiltonian, it comes directly from the solution in one cell and all the translational symmetries of the system. However, we can also approach the problem by considering a finite system of N atoms in a loop. We can explicitly build the TB Hamiltonian for this finite system and solve for its eigenvalues and eigenvectors. For a system of 6 atoms this is equivalent to the Huckel model of benzene. As the number of atoms in the loop goes to infinity, we expect this problem to converge to the expression above.

**Questions**

Before you run any simulation, answer the following question(s):

1. Consider the explicit TB/Huckel model for N atoms in a chain. How big is the associated Hamiltonian? How many states do you expect to find as the solution of the QM problem?
2. If your system has $N$ cells of size $a$, how many cosine or sine waves with period larger than $a$ can you fit in your system?
3. In the limit of $N$ becoming very large, how many electronic states do you expect your system will have? 

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

4. While the results (states and eigenvectors) will generally change as the value of $N$ is changed, some states tend to be always present and have higher symmetry than others. These special states are associated with special points in the reciprocal lattice. For the case of a 1D lattice, the two main high-symmetry points are the center of the Brillouin zone (also konwn as gamma point or $\Gamma$) and the edge of the Brillouin zone (X). How are the solutions of the 1D TB Hamiltonian at these points? 
5. What is the shape (dispersion) of the total energy of the system as a function of the Brillouin zone vector close to $\Gamma$? 
6. Compare the above result with the result for a free electron and, using the following definition, compare the effective mass, $m^{*}$,of the TB electron at $\Gamma$ with the one of the free electron:
$$\frac{1}{m^{*}}=\frac{1}{\hbar^2} \frac{d^2E}{dk^2}$$


In [ ]:
# @title Simulation Parameters { display-mode: "form" }
N = 10 # @param {type:"integer"}
epsilon = 0.0 # @param {type:"number"}
t = -1.0 # @param {type:"number"}
a = 1.0 # @param {type:"number"}
periodic = True # @param {type:"boolean"}

# periodic chain
E_per, V_per = solve_finite_chain(N=N, epsilon=epsilon, t=t, periodic=True)
k_signed, V_chiral = label_k_via_translation(E_per, V_per, a=1.0, energy_tol=1e-12)

# --- Setup GridSpec with 2 rows, 3 columns ---
fig = plt.figure(figsize=(14, 8))
gs = GridSpec(2, 3, height_ratios=[2, 2.5])  # 2 rows, 3 cols

groups = group_degenerate_energies(E_per, tol=1e-12)

# Top-left: ladder plot
ax0 = fig.add_subplot(gs[0, 0])
for group in groups:
    y = E_per[group]
    n = len(group)
    x_center = 0.0
    spacing = 0.2
    if n == 1:
        x_positions = [x_center]
    else:
        offsets = np.linspace(-(n - 1) / 2, (n - 1) / 2, n) * spacing
        x_positions = x_center + offsets
    for xi, yi in zip(x_positions, y):
        ax0.hlines(yi, xi - 0.1, xi + 0.1, color="black", linewidth=2)
        ax0.plot(xi, yi, 'ko')

ax0.set_title("Energy levels (finite ring)")
ax0.set_ylabel("Energy (eV)")
ax0.set_xticks([])
ax0.set_xlim(-1, 1)
ax0.grid(True, axis="y", linestyle="--", alpha=0.5)

# Top-middle and right: dispersion plot
ax1 = fig.add_subplot(gs[0, 1:3])  # span columns 1 and 2
k_grid = np.linspace(-np.pi, np.pi, 400)
E_k = tb_dispersion_1d(k_grid, epsilon=epsilon, t=t, a=a)
order = np.argsort(k_signed)
ax1.plot(k_grid, E_k, label=r"$E(k) = \varepsilon + 2t\cos(ka)$", color='gray')
ax1.plot(k_signed[order], E_per[order], 'o', color='blue', label='Finite chain eigenstates')
ax1.set_xlabel(r"$k$ (1/a)")
ax1.set_ylabel("Energy (eV)")
ax1.set_title("Dispersion + finite-chain states")
ax1.legend()
ax1.grid(True)
ax1.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
ax1.set_xticklabels([r"$-\pi$", r"$-\frac{\pi}{2}$", "0", r"$\frac{\pi}{2}$", r"$\pi$"])

# Bottom row: eigenstates
indices_to_plot = [0, N//2, N-1]
titles = [r"$\psi_0$ (lowest)", rf"$\psi_{{{N//2}}}$ (middle)", rf"$\psi_{{{N-1}}}$ (highest)"]

for i, (idx, title) in enumerate(zip(indices_to_plot, titles)):
    ax = fig.add_subplot(gs[1, i])
    psi = V_chiral[:, idx]
    colors = ['blue' if v.real >= 0 else 'red' for v in psi]
    ax.bar(range(N), psi.real, color=colors)
    ax.set_title(title)
    ax.set_xlabel("Site index")
    ax.set_ylabel(r"Re[$\psi_j$]")
    ax.set_ylim(-1.1, 1.1)
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()


In [ ]:
# @title Visualization of Individual States { display-mode: "form" }
eigenstate = 6 # @param {type:"integer"}
form = 'bloch' # @param ['real', 'bloch']
if eigenstate < 0 or eigenstate >= N:
    raise ValueError(f"eigenstate must be between 0 and {N-1}")
# Plot selected eigenstate
if form == 'bloch':
    visualize_state_card_periodic(E_per, V_chiral, index=eigenstate, a=a, k=float(k_signed[eigenstate]))
else:
    visualize_state_card_periodic(E_per, V_per, index=eigenstate, a=a)


**Homework Assignment**

Use the simplified TB (i.e. Huckel) model to derive the band structure of graphene, a 2D honeycomb lattice of carbon atoms, assuming two atoms per cell and only considering the $p_z$ orbitals of carbon. More details on the steps of the assignment can be found in the provided pdf.